In [1]:

# from duong.gen_angelic_st import GenAngelic
from minh.deepseek import chat


promptInputPath = "./duong/init_prompt_angelic_4.1.txt"
# chat = GenAngelic().init()
class Answerer:
    def __init__(self, debug=False):
        self.debug = debug

    def answer(self, article, query):
        reply = chat(article, query, prompt_input_path = promptInputPath)
        if self.debug:
            print("======== Generate Angelic structure and Query Entailment: ========\n", reply)

text = ""
with open("./test.txt", "r") as f:
    text = "".join(f.readlines())
answerer = Answerer(debug=True)
answer = answerer.answer(text, "In cases where an individual rescues another person from getting hit by a car by pushing that person out of the way, causing the person's luxury kimono to get dirty, the rescuer does not have to compensate damages for the kimono.")
print(answer)


======== Generate Angelic structure and Query Entailment: ========
 To construct the ANGELIC structure based on the provided legal provisions and then analyze the given query in relation to this structure, we follow the step-by-step approach outlined:

### 1. Identify the Root Node (R)

The root node is determined by answering what legal conclusion or verdict the article seeks to establish or refute. In Article 698, the legal conclusion is that a manager who engages in benevolent intervention in another's business to prevent imminent danger is not liable for damages resulting from this action unless they acted in bad faith or with gross negligence.

- **R1:** A manager is not liable for damages resulting from benevolent intervention unless acting in bad faith or with gross negligence.

### 2. Derive Abstract Factors (A)

The abstract factors are the key legal principles that either substantiate or undermine the legal conclusion. For Article 698, these include:

- **A1:** Benevolent int

In [4]:
from extractor.extractor import MDAExtractor
# from duong.gen_angelic_st import GenAngelic
from reasoner.reasoner import Reasoner
import dspy

# chat = GenAngelic().init()
class Answerer:
    def __init__(self, reply, debug=False):
        self.extractor = MDAExtractor()
        self.reasoner = Reasoner()
        self.debug = debug
        self.reply = reply

    def answer(self, article, query):
        facts = self.extractor.forward(text=self.reply)
        if self.debug:
            print("======= Fact Extractor step ========\n")
            print("Facts:\n", facts)
            print("\n")
        # theory_2
        return self.reasoner.reason(facts)



dspy.settings.configure(
    lm=dspy.LM(
        model="ollama_chat/llama3.3",  # Changed to include provider prefix
        api_base="http://localhost:11435",
        max_tokens=20000,
    )
)
text = ""
reply = ""
with open("./test_reply.txt", "r") as f:
    reply = "".join(f.readlines())
answerer = Answerer(reply=reply, debug=True)

answer = answerer.answer(text, "In cases where an individual rescues another person from getting hit by a car by pushing that person out of the way, causing the person's luxury kimono to get dirty, the rescuer does not have to compensate damages for the kimono.")
print(answer)


======= Fact Extractor step ========

Facts:
 link(support(node(afactor(1)),node(root(1)))).link(support(node(afactor(2)),node(root(1)))).link(support(node(bfactor(1)),node(afactor(1)))).link(support(node(bfactor(2)),node(afactor(1)))).link(support(node(bfactor(3)),node(afactor(2)))).entail(query, node(bfactor(2))).entail(query, node(bfactor(3))).agree(query, node(root(1))).


True


In [1]:
import json

from answerer import Answerer, ExtractorException

import dspy
dspy.settings.configure(
    lm=dspy.LM(
        model="ollama_chat/llama3.3",  # Changed to include provider prefix
        api_base="http://localhost:11435",
        max_tokens=20000
    )
)
import logging

logging.getLogger("requests").setLevel(logging.CRITICAL)

cases = []
with open('train_as_test.json') as f:
    cases = json.load(f)

module = Answerer(debug=True)
print("Generate finetuning data for extractor...")
data = []
# data_count = 2
for i, c in enumerate(cases[11:]):
    query=c["query"]
    articles=c["paragraphs"]
    expected=c["label"]
    if expected == "N": continue
    failed_answer = None
    while True:
        try:
            chat, l= module.retry_chat(article=articles, query=query, failed_answer=failed_answer)
            l = "Y" if l else "N"
            if l == expected:
                data.append({"articles:":articles, "query:":query, "Angelic structure: ": chat})
                print("Generated for case", i)
                with open(f"finetuning_chat_data_2_Mar3_label=Y.json", "w") as f:
                    f.writelines(json.dumps(data))
                break
            else:
                print("retrying... wrong answer")
                failed_answer = f"{chat}"
                continue
        except ExtractorException as e:
            print("retrying...  failed")
            failed_answer = f"The following answer failed because it's not the right Clingo syntax: {e.args[0]}"
            continue
        except Exception as e:
            print(e)
            continue
    if len(data) == 5: 
        # with open(f"finetuning_extractor_data_{data_count}.json", "w") as f:
        #     f.writelines(json.dumps(data))
        # data_count+= 1
        data = []
        break

print("Done extractor...")

# print("Generate finetuning data for chat...")
# data = []
# data_count = 0
# for i, c in enumerate(cases):
#     query=c["query"]
#     articles=c["paragraphs"]
#     expected=c["label"]

#     failed_answer = ""
#     while True:
#         try:
#             chat, l= module.retry_chat(article=articles, query=query, failed_answer=failed_answer)
#             l = "Y" if l else "N"
#             if l == expected:
#                 data.append({"query": query, "articles": articles, "chat": chat})
#                 print("Generated for case", i)
#                 break
#             else:
#                 print("retrying... wrong answer")
#                 failed_answer = chat 
#                 continue
#         except ExtractorException as e:
#             print("retrying...  failed")
#             failed_answer = e.args[0]
#             continue
#         except:
#             raise
#     if len(data) == 10: 
#         with open(f"finetuning_chat_data_{data_count}.json", "w") as f:
#             f.writelines(json.dumps(data))
#         data_count+= 1
#         data = []
# if len(data) > 0:
#     with open(f"finetuning_chat_data_{data_count}.json", "w") as f:
#         f.writelines(json.dumps(data))
# print("Done chat...")

/home/anguyen/miniconda3/envs/coliee2025/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Generate finetuning data for extractor...
======== Angelic extraction step: ======== 
 To construct the ANGELIC structure based on the provided legal provisions, let's follow a step-by-step approach:

### Step 1: Identify the Root Node (R)
The root node represents the main legal conclusion or verdict that the article seeks to establish. In this case, there are three distinct conclusions related to actions for maintenance of possession, preservation of possession, and recovery of possession. However, focusing on the query provided, "An action for recovery of possession must be brought within one year from the time when possession was unlawfully usurped," we can identify a root node as:
- **R1**: An action for recovery of possession must be filed within a specific timeframe after being dispossessed.

### Step 2: Derive Abstract Factors (A)
Abstract factors are key legal principles or conditions that validate or invalidate the conclusion. For R1, relevant abstract factors include:
- **A1*

KeyboardInterrupt: 

In [9]:
import json

def fix_json_file(input_file, output_file):
    with open(input_file, 'r') as file:
        content = file.read()

    # Split the content into instances
    instances = content.split('}{')

    fixed_instances = ['{' + instance + '}' for instance in instances]
    fixed_content = '[\n' + ',\n'.join(fixed_instances) + '\n]'

    # Write the fixed content to the output file
    with open(output_file, 'w') as file:
        file.write(fixed_content)

# File paths
input_file = '/home/anguyen/KLAP_COLIEE2025/finetuning_chat_data_2_Mar3_correct_predict.json'
output_file = '/home/anguyen/KLAP_COLIEE2025/finetuning_chat_data_2_Mar3__correct_predict_fixed.json'

# Fix the JSON file
fix_json_file(input_file, output_file)

In [10]:
import json

def count_instances_in_json_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    
    # Count the number of instances
    num_instances = len(data)
    print(f"Number of instances: {num_instances}")

# File path
file_path = '/home/anguyen/KLAP_COLIEE2025/finetuning_chat_data_2_Mar3__correct_predict_fixed.json'

# Count the instances in the JSON file
count_instances_in_json_file(file_path)

Number of instances: 97
